In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [11]:
import json
labels_map = json.load(open("label_maps.json"))


In [12]:
class FlowerCNN(nn.Module):
    def __init__(self, num_classes=102):
        super(FlowerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.6)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.max_pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.max_pool(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.max_pool(x)

        x = self.conv4(x)
        x = self.relu(x)
        x = self.bn4(x)
        x = self.max_pool(x)

        x = self.conv5(x)
        x = self.relu(x)
        x = self.bn5(x)
        x = self.max_pool(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [13]:
'''
34% accuracy
learning_rate = 0.0001 # 0.0001 = 41.5% accuracy
batch_size = 16
num_epochs = 50
momentum = 0.9
weight_decay = 1e-4

35.4
learning_rate = 0.0001 # 0.0001 = 41.5% accuracy
batch_size = 16
num_epochs = 50
momentum = 0.9
weight_decay = 1e-3#

learning_rate = 0.0001 # 0.0001 = 41.5% accuracy
batch_size = 8
num_epochs = 50
momentum = 0.9
weight_decay = 1e-3

36.5
learning_rate = 0.00008 # 0.0001 = 41.5% accuracy
batch_size = 8
num_epochs = 50
momentum = 0.9
weight_decay = 1e-3

'''

'\n34% accuracy\nlearning_rate = 0.0001 # 0.0001 = 41.5% accuracy\nbatch_size = 16\nnum_epochs = 50\nmomentum = 0.9\nweight_decay = 1e-4\n\n35.4\nlearning_rate = 0.0001 # 0.0001 = 41.5% accuracy\nbatch_size = 16\nnum_epochs = 50\nmomentum = 0.9\nweight_decay = 1e-3#\n\nlearning_rate = 0.0001 # 0.0001 = 41.5% accuracy\nbatch_size = 8\nnum_epochs = 50\nmomentum = 0.9\nweight_decay = 1e-3\n\n36.5\nlearning_rate = 0.00008 # 0.0001 = 41.5% accuracy\nbatch_size = 8\nnum_epochs = 50\nmomentum = 0.9\nweight_decay = 1e-3\n\n'

In [14]:
# HYPERPARAMETERS
learning_rate = 0.0002 # 0.0001 = 41.5% accuracy
batch_size = 8
num_epochs = 5
momentum = 0.9
weight_decay = 1e-3


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [15]:
import torchvision
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset

train_set = torchvision.datasets.Flowers102(root="data", download=True, transform=transform)
test_set = torchvision.datasets.Flowers102(root="data", download=True, split="test", transform=transform)
val_set = torchvision.datasets.Flowers102(root="data", download=True, split="val", transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


In [16]:
model = FlowerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

In [17]:
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%")

# Testing loop


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x25088 and 2048x1024)

In [ ]:
import csv
model.to(device)
# Testing loop
model.eval()
class_correct = [0 for _ in range(102)]
class_total = [0 for _ in range(102)]
class_accuracies = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += (predicted[i] == label).item()
            class_total[label] += 1

    accuracy = 100 * sum(class_correct) / sum(class_total)
    print(f"Test Accuracy: {accuracy:.2f}%")

    for i in range(102):
        if class_total[i] > 0:
            class_accuracy = 100 * class_correct[i] / class_total[i]
            class_accuracies.append((labels_map[str(i)], f"{class_accuracy}%"))

sorted_class_accuracies = sorted(class_accuracies, key=lambda x: x[1], reverse=True)

with open("class_accuracies.csv", "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Class", "Accuracy"])

    for class_info in sorted_class_accuracies:
        class_index, class_accuracy = class_info
        csv_writer.writerow([class_index, class_accuracy])

RuntimeError: stack expects each tensor to be equal size, but got [3, 500, 523] at entry 0 and [3, 500, 666] at entry 1

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# model = FlowerCNN()
# dummy_input = torch.zeros((1, 3, 64, 64))  # Create a dummy input with the same dimensions as your input images
# writer = SummaryWriter(log_dir='runs/flower_cnn')

# # Add the model graph to TensorBoard
# writer.add_graph(model, dummy_input)
# writer.close()

In [ ]:
import time
print("Saving model...")
torch.save(model.state_dict(), f"./models/model_42.pth")
print("Model saved!")


Saving model...
Model saved!


In [ ]:
# # Testing loop
# model.to(device)
# model.eval()
# class_correct = [0 for _ in range(102)]
# class_total = [0 for _ in range(102)]

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         outputs = model(inputs)
#         _, predicted = torch.max(outputs.data, 1)

#         for i in range(labels.size(0)):
#             label = labels[i]
#             class_correct[label] += (predicted[i] == label).item()
#             class_total[label] += 1

#     accuracy = 100 * sum(class_correct) / sum(class_total)
#     print(f"Test Accuracy: {accuracy:.2f}%")

#     for i in range(102):
#         if class_total[i] > 0:
#             class_accuracy = 100 * class_correct[i] / class_total[i]
#             print(f"Class {i}: Accuracy: {class_accuracy:.2f}% ({class_correct[i]}/{class_total[i]})")
#         else:
#             print(f"Class {i}: No samples")
